# Folium Test 16/17

In [66]:
# Imports
import pandas as pd
import functions as fc
import numpy as np
import matplotlib.pyplot as plt
import folium

sales_frame = fc.read_csv("data/nyc-rolling-sales2.csv")

# Filtering sales_frame for ZIP CODES > 0 and changing datatype to string
zipcodes = sales_frame[sales_frame["ZIP CODE"]>0]
zipcodes["ZIP"] = zipcodes["ZIP CODE"].astype(str)
boroughs = zipcodes[["ZIP", "BOROUGH"]]
boroughs.drop_duplicates("ZIP", inplace=True)

# Getting Latitude and Longitude Values for each ZIP CODE
us_zipcodes = pd.read_csv("data/US Zip Codes from 2013 Government Data.txt", delimiter= ",", dtype=str)
zipcodes_agg=pd.merge(zipcodes.groupby("ZIP").agg(np.mean), us_zipcodes, how="left", on="ZIP")
zipcodes_agg = pd.merge(zipcodes_agg, boroughs, how="left", on="ZIP")
print(zipcodes_agg)

# Dropping all nan values for further processing
zipcodes_agg["LAT"].replace("", np.nan, inplace=True)
zipcodes_agg.dropna(subset=["LAT"], inplace=True)

# Checking new dataframe
fc.to_csv("data/foliumtest.csv", zipcodes_agg)

from folium.plugins import MarkerCluster
map = folium.Map(location=[40.693943, -73.985880], default_zoom_start=12)
map.choropleth(geo_data="data/nyc-zip-code-tabulation-areas-polygons.geojson",
             data=zipcodes_agg, # my dataset
             columns=["ZIP", "SALE PRICE"], # zip code is here for matching the geojson zipcode, sales price is the column that changes the color of zipcode areas
             key_on='feature.properties.postalCode', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='SALE PRICE')


marker_cluster = MarkerCluster().add_to(map)
for i in range(zipcodes_agg.shape[0]):
    location = [zipcodes_agg['LAT'][i],zipcodes_agg['LNG'][i]]
    tooltip = "Zipcode:{}<br> Borough: {}<br> Click for more".format(zipcodes_agg["ZIP"][i], zipcodes_agg['BOROUGH'][i])
    folium.Marker(location,
                  popup="""<i>Mean sales price: </i> <br> <b>${}</b> <br>
                  <i>mean total units: </i><b><br>{}</b><br>
                  <i>mean square feet: </i><b><br>{}</b><br>""".format(round(zipcodes_agg['SALE PRICE'][i],2), round(zipcodes_agg['TOTAL UNITS'][i],2), round(zipcodes_agg['GROSS SQUARE FEET'][i],2)),
                  tooltip=tooltip).add_to(marker_cluster)

map
map.save("graphics/nyc_map.html")


       ZIP         BLOCK          LOT  ZIP CODE  RESIDENTIAL UNITS  \
0    10001    769.426471   550.274510   10001.0           7.303922   
1    10002    328.079268   359.317073   10002.0           2.106707   
2    10003    672.133005   531.520936   10003.0           1.391626   
3    10004     18.894737  1776.642105   10004.0           1.221053   
4    10005     32.226131  1234.095477   10005.0           4.929648   
..     ...           ...          ...       ...                ...   
180  11436  12035.474359    69.967949   11436.0           2.407051   
181  11691  15698.602299    93.032184   11691.0           1.439080   
182  11692  16028.388535    52.732484   11692.0           1.738854   
183  11693  15544.753521   195.436620   11693.0           1.183099   
184  11694  16235.926740   235.578755   11694.0           1.937729   

     COMMERCIAL UNITS  TOTAL UNITS  LAND SQUARE FEET  GROSS SQUARE FEET  \
0            1.710784     9.088235        308.899743        2461.706351   
1        

C:\Users\Dome\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Dome\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Dome\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


KeyError: 41